# Imports

In [1]:
import sys, torch, pickle, csv
import numpy as np
import scipy as sp
sys.path.append('/lfs/1/danfu/metal')
sys.path.append('/lfs/1/danfu/sequential_ws')
from metal.metrics import metric_score
from torch.nn.functional import normalize
from DP.label_model import *
import pandas as pd
from metal.label_model.baselines import MajorityLabelVoter

In [2]:
device = torch.device('cuda:0')

# Load data

In [3]:
L_train_path = '/lfs/1/danfu/esper/app/data/shot_detection_weak_labels/L_train_100_windows_high_pre.npz'
L_dev_path = '/lfs/1/danfu/esper/app/data/shot_detection_weak_labels/L_dev_windows_high_pre.npz'
Y_dev_path = '/lfs/1/danfu/esper/app/data/shot_detection_weak_labels/Y_dev_windows_high_pre.npy'

stride = 1
L_train_raw = sp.sparse.load_npz(L_train_path).todense()[::stride]
L_dev_raw = sp.sparse.load_npz(L_dev_path).todense()
Y_dev_raw = np.load(Y_dev_path)

T = 5

L_train = torch.FloatTensor(L_train_raw[:L_train_raw.shape[0] - (L_train_raw.shape[0] % T)]).to(device)
L_dev = torch.FloatTensor(L_dev_raw[:L_dev_raw.shape[0] - (L_dev_raw.shape[0] % T)]).to(device)
Y_dev = torch.FloatTensor(Y_dev_raw[:Y_dev_raw.shape[0] - (Y_dev_raw.shape[0] % T)]).to(device)
m_per_task = L_train.size(1)
n_frames_train = L_train.size(0)
n_patients_train = n_frames_train//T
n_frames_dev = L_dev.size(0)
n_patients_dev = n_frames_dev//T

# MRI_data_naive = {'Li_train': (L_train.unsqueeze(2) == torch.FloatTensor([-1,1,0]).to(device).unsqueeze(0).unsqueeze(0)).argmax(2),
#                   'Li_dev': (L_dev.unsqueeze(2) == torch.FloatTensor([-1,1,0]).to(device).unsqueeze(0).unsqueeze(0)).argmax(2),
#                   'R_dev': (Y_dev.unsqueeze(1) == torch.FloatTensor([-1,1]).to(device).unsqueeze(0)).argmax(1),
#                   'm':m_per_task, 'T':1,
#                  }

# don't need to transform the raw data
MRI_data_naive = {'Li_train': L_train.long().to(device),
                  'Li_dev': L_dev.long().to(device),
                  'R_dev': Y_dev.long().to(device),
                  'm':m_per_task, 'T':1,
                 }
MRI_data_naive['class_balance'] = normalize((MRI_data_naive['R_dev'].unsqueeze(1)==torch.arange(2, device=device).unsqueeze(0)).sum(0).float(), 
                                            dim=0, p=1)
MRI_data_temporal = {'Li_train': MRI_data_naive['Li_train'].view(n_patients_train, (m_per_task*T)),
                     'Li_dev': MRI_data_naive['Li_dev'].view(n_patients_dev, (m_per_task*T)),
                     'R_dev': MRI_data_naive['R_dev']*(2**T-1),
                     'm': m_per_task * T, 'T': T,
                    } 
MRI_data_temporal['class_balance'] = normalize((MRI_data_temporal['R_dev'].unsqueeze(1)==torch.arange(2**T, device=device).unsqueeze(0)).sum(0).float(), 
                                                dim=0, p=1)

# Naive model

In [121]:
naive_model = DPLabelModel(m=m_per_task, 
                           T=1,
                           edges=[],
                           coverage_sets=[[0,]]*m_per_task,
                           mu_sharing=[[i,] for i in range(m_per_task)],
                           phi_sharing=[],
                           device=device,
                           class_balance=torch.tensor([0.4, 0.6]).to(device), 
                           seed=0)
optimize(naive_model, L_hat=MRI_data_naive['Li_train'], num_iter=10000,
         lr=4.087885261759692e-05, momentum=0.9, clamp=True, seed=123)

TypeError: __init__() got an unexpected keyword argument 'l2'

In [116]:
R_pred = naive_model.predict(MRI_data_naive['Li_dev']) + 1
for metric in ['accuracy', 'f1', 'recall', 'precision']:
    score = metric_score(MRI_data_naive['R_dev'].cpu(), R_pred.cpu(), metric)
    print(f"{metric.capitalize()}: {score:.3f}")

Accuracy: 0.394
F1: 0.143
Recall: 0.288
Precision: 0.095


In [117]:
R_pred_flipped = torch.where(torch.tensor(R_pred.cpu())==2.,torch.tensor(1.),torch.tensor(2.))

In [123]:
R_pred_flipped[:100]

tensor([2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 2., 2., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 2., 2., 2.])

In [118]:
for metric in ['accuracy', 'f1', 'recall', 'precision']:
    score = metric_score(MRI_data_naive['R_dev'].cpu(), R_pred_flipped, metric)
    print(f"{metric.capitalize()}: {score:.3f}")

Accuracy: 0.606
F1: 0.389
Recall: 0.712
Precision: 0.268


# Majority Vote

In [71]:
mv = MajorityLabelVoter(seed=123)
mv_pred = mv.predict(MRI_data_naive['Li_dev'])
for metric in ['accuracy', 'f1', 'recall', 'precision']:
    score = metric_score(MRI_data_naive['R_dev'].cpu(), mv_pred, metric)
    print(f"{metric.capitalize()}: {score:.3f}")

Accuracy: 0.923
F1: 0.778
Recall: 0.772
Precision: 0.785


# Timeseries Model

In [10]:
%%time
best = None
for iterations in [1, 10, 50, 100, 500, 1000, 5000]:
    print(iterations)
    max_seed = 10
    temporal_models = [None,]*max_seed
    for seed in range(max_seed):
        markov_model = DPLabelModel(m=m_per_task*T, 
                                    T=T,
                                    edges=[(i,i+m_per_task) for i in range((T-1)*m_per_task)],
                                    coverage_sets=[[t,] for t in range(T) for _ in range(m_per_task)],
                                    mu_sharing=[[t*m_per_task+i for t in range(T)] for i in range(m_per_task)],
                                    phi_sharing=[[(t*m_per_task+i, (t+1)*m_per_task+i)
                                                  for t in range(T-1)] for i in range(m_per_task)],
                                    device=device,
                                    # class_balance=MRI_data_temporal['class_balance'],
                                    seed=seed)
        optimize(markov_model, L_hat=MRI_data_temporal['Li_train'], num_iter=iterations,
                 lr=1e-5, momentum=0.8, clamp=True, 
                 verbose=False, seed=seed)
        temporal_models[seed] = markov_model
    
    for seed, model in enumerate(temporal_models):
        Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
        R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

        #find sequence label config. with highest prob.
        config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
        R_pred_config = model.feasible_y[config_index]
        R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

        #for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
        R_pred_probs = torch.FloatTensor(R_pred_config.shape)
        for idx in range(R_pred_config.shape[0]):
            R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

        R_pred_probs = R_pred_probs.numpy()
        R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]

        Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
        R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

        #find sequence label config. with highest prob.
        config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
        R_pred_config = model.feasible_y[config_index]
        R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

        #for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
        R_pred_probs = torch.FloatTensor(R_pred_config.shape)
        for idx in range(R_pred_config.shape[0]):
            R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

        R_pred_probs = R_pred_probs.numpy()
        R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]
        R_pred_frame_label = np.round(R_pred_probs.ravel())
        R_pred_frame_label[R_pred_frame_label == 0.] = 2.

        scores = [iterations, seed, model]
        for metric in ['accuracy', 'f1', 'recall', 'precision']:
            score = metric_score(Y_dev.cpu(), R_pred_frame_label, metric)
            print(f"{metric.capitalize()}: {score:.3f}")
            
            scores.append(score)
        
        if best == None or scores[4] > best[4]:
            best = scores
        print()

print(best)

1
Accuracy: 0.335
F1: 0.146
Recall: 0.322
Precision: 0.094

Accuracy: 0.610
F1: 0.316
Recall: 0.511
Precision: 0.228

Accuracy: 0.545
F1: 0.239
Recall: 0.407
Precision: 0.170

Accuracy: 0.418
F1: 0.212
Recall: 0.446
Precision: 0.139

Accuracy: 0.364
F1: 0.222
Recall: 0.517
Precision: 0.142

Accuracy: 0.471
F1: 0.236
Recall: 0.464
Precision: 0.158

Accuracy: 0.546
F1: 0.327
Recall: 0.628
Precision: 0.221

Accuracy: 0.288
F1: 0.191
Recall: 0.477
Precision: 0.119

Accuracy: 0.448
F1: 0.201
Recall: 0.395
Precision: 0.135

Accuracy: 0.497
F1: 0.236
Recall: 0.442
Precision: 0.161

10
Accuracy: 0.449
F1: 0.184
Recall: 0.354
Precision: 0.125

Accuracy: 0.376
F1: 0.247
Recall: 0.583
Precision: 0.157

Accuracy: 0.473
F1: 0.206
Recall: 0.387
Precision: 0.140

Accuracy: 0.578
F1: 0.318
Recall: 0.560
Precision: 0.222

Accuracy: 0.528
F1: 0.229
Recall: 0.398
Precision: 0.160

Accuracy: 0.615
F1: 0.242
Recall: 0.349
Precision: 0.185

Accuracy: 0.518
F1: 0.068
Recall: 0.100
Precision: 0.052

Accuracy:

In [83]:
best[2]

DPLabelModel()

In [85]:
model = best[2]
Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

#find sequence label config. with highest prob.
config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
R_pred_config = model.feasible_y[config_index]
R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

#for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
R_pred_probs = torch.FloatTensor(R_pred_config.shape)
for idx in range(R_pred_config.shape[0]):
    R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

R_pred_probs = R_pred_probs.numpy()
R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]

Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

#find sequence label config. with highest prob.
config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
R_pred_config = model.feasible_y[config_index]
R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

#for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
R_pred_probs = torch.FloatTensor(R_pred_config.shape)
for idx in range(R_pred_config.shape[0]):
    R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

R_pred_probs = R_pred_probs.numpy()
R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]
R_pred_frame_label = np.round(R_pred_probs.ravel())
R_pred_frame_label[R_pred_frame_label == 0.] = 2.

scores = [iterations, seed, model]
for metric in ['accuracy', 'f1', 'recall', 'precision']:
    score = metric_score(Y_dev.cpu(), R_pred_frame_label, metric)
    print(f"{metric.capitalize()}: {score:.3f}")

    scores.append(score)

if best == None or scores[4] > best[4]:
    best = scores
print()

Accuracy: 0.606
F1: 0.389
Recall: 0.712
Precision: 0.268



In [94]:
R_pred_frame_label[:100]

array([2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 2., 2., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2.],
      dtype=float32)

In [101]:
R_pred_frame_label_flipped = torch.where(torch.tensor(R_pred_frame_label)==2.,torch.tensor(1.),torch.tensor(2.))

In [95]:
Y_dev[:100]

tensor([1., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 1., 1., 2., 2., 1., 1., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 1., 1., 2., 1., 1., 2., 2., 1., 1., 2., 1., 1., 2.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        1., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 1., 1., 2., 2., 2.], device='cuda:0')

In [91]:
R_pred_frame.size()

torch.Size([1465, 32])

In [92]:
R_pred_frame[:10]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 

In [93]:
model.feasible_y

tensor([[-1, -1, -1, -1, -1],
        [-1,  1, -1, -1, -1],
        [ 1, -1, -1, -1, -1],
        [ 1,  1, -1, -1, -1],
        [-1, -1,  1, -1, -1],
        [-1,  1,  1, -1, -1],
        [ 1, -1,  1, -1, -1],
        [ 1,  1,  1, -1, -1],
        [-1, -1, -1,  1, -1],
        [-1,  1, -1,  1, -1],
        [ 1, -1, -1,  1, -1],
        [ 1,  1, -1,  1, -1],
        [-1, -1,  1,  1, -1],
        [-1,  1,  1,  1, -1],
        [ 1, -1,  1,  1, -1],
        [ 1,  1,  1,  1, -1],
        [-1, -1, -1, -1,  1],
        [-1,  1, -1, -1,  1],
        [ 1, -1, -1, -1,  1],
        [ 1,  1, -1, -1,  1],
        [-1, -1,  1, -1,  1],
        [-1,  1,  1, -1,  1],
        [ 1, -1,  1, -1,  1],
        [ 1,  1,  1, -1,  1],
        [-1, -1, -1,  1,  1],
        [-1,  1, -1,  1,  1],
        [ 1, -1, -1,  1,  1],
        [ 1,  1, -1,  1,  1],
        [-1, -1,  1,  1,  1],
        [-1,  1,  1,  1,  1],
        [ 1, -1,  1,  1,  1],
        [ 1,  1,  1,  1,  1]], device='cuda:0')

In [88]:
Y_dev[-10:]

tensor([1., 1., 2., 2., 2., 2., 2., 2., 2., 1.], device='cuda:0')

# Timeseries Model Class Balance

In [4]:
valid_target = Y_dev.long()
T = 5

feasible_y = np.array([[-1, -1, -1, -1, -1],
        [-1,  1, -1, -1, -1],
        [ 1, -1, -1, -1, -1],
        [ 1,  1, -1, -1, -1],
        [-1, -1,  1, -1, -1],
        [-1,  1,  1, -1, -1],
        [ 1, -1,  1, -1, -1],
        [ 1,  1,  1, -1, -1],
        [-1, -1, -1,  1, -1],
        [-1,  1, -1,  1, -1],
        [ 1, -1, -1,  1, -1],
        [ 1,  1, -1,  1, -1],
        [-1, -1,  1,  1, -1],
        [-1,  1,  1,  1, -1],
        [ 1, -1,  1,  1, -1],
        [ 1,  1,  1,  1, -1],
        [-1, -1, -1, -1,  1],
        [-1,  1, -1, -1,  1],
        [ 1, -1, -1, -1,  1],
        [ 1,  1, -1, -1,  1],
        [-1, -1,  1, -1,  1],
        [-1,  1,  1, -1,  1],
        [ 1, -1,  1, -1,  1],
        [ 1,  1,  1, -1,  1],
        [-1, -1, -1,  1,  1],
        [-1,  1, -1,  1,  1],
        [ 1, -1, -1,  1,  1],
        [ 1,  1, -1,  1,  1],
        [-1, -1,  1,  1,  1],
        [-1,  1,  1,  1,  1],
        [ 1, -1,  1,  1,  1],
        [ 1,  1,  1,  1,  1]])

feasible_y[feasible_y==-1] = 0
feasible_y = feasible_y.tolist()
possibilities = list(map(lambda l : ''.join(map(str,l)), feasible_y))

class_balance = np.empty(2 ** T)
#compute class balance from dev set and use laplace smoothing

valid_target_copy = np.copy(valid_target)
valid_target_copy[valid_target_copy == 2] = 0

assert len(valid_target_copy) % T == 0
num_windows = len(valid_target_copy) / T

freq = {}
for i in range(0, len(valid_target_copy), T):
    s = ''.join(map(str,valid_target_copy[i:i+T]))
    if s in freq:
        freq[s] += 1
    else:
        freq[s] = 1

for i in range(len(class_balance)):
    if possibilities[i] in freq and freq[possibilities[i]] > 5:
        class_balance[i] = (freq[possibilities[i]] + 1) / (num_windows + len(possibilities))
    else:
        class_balance[i] = 0.

In [6]:
freq

{'11000': 95,
 '00001': 91,
 '10000': 96,
 '00000': 792,
 '11001': 8,
 '01101': 16,
 '10011': 12,
 '00110': 91,
 '01100': 97,
 '00011': 94,
 '01111': 10,
 '11011': 5,
 '00111': 10,
 '01110': 4,
 '11111': 6,
 '11100': 11,
 '10001': 13,
 '10111': 1,
 '11110': 6,
 '10110': 7}

In [5]:
class_balance

array([0.52972612, 0.        , 0.06479626, 0.06412826, 0.        ,
       0.06546426, 0.        , 0.00801603, 0.        , 0.        ,
       0.        , 0.        , 0.06145625, 0.        , 0.00534402,
       0.00467602, 0.06145625, 0.        , 0.00935204, 0.00601202,
       0.        , 0.01135605, 0.        , 0.        , 0.06346025,
       0.        , 0.00868403, 0.        , 0.00734803, 0.00734803,
       0.        , 0.00467602])

In [155]:
%%time
best_class_balance = None
for iterations in [1, 10, 50, 100, 500, 1000, 5000]:
    print(iterations)
    max_seed = 10
    temporal_models = [None,]*max_seed
    for seed in range(max_seed):
        markov_model = DPLabelModel(m=m_per_task*T, 
                                    T=T,
                                    edges=[(i,i+m_per_task) for i in range((T-1)*m_per_task)],
                                    coverage_sets=[[t,] for t in range(T) for _ in range(m_per_task)],
                                    mu_sharing=[[t*m_per_task+i for t in range(T)] for i in range(m_per_task)],
                                    phi_sharing=[[(t*m_per_task+i, (t+1)*m_per_task+i)
                                                  for t in range(T-1)] for i in range(m_per_task)],
                                    device=device,
                                    class_balance=torch.tensor(class_balance).float().to(device),
                                    seed=seed)
        optimize(markov_model, L_hat=MRI_data_temporal['Li_train'], num_iter=iterations,
                 lr=1e-5, momentum=0.8, clamp=True, 
                 verbose=False, seed=seed)
        temporal_models[seed] = markov_model
    
    for seed, model in enumerate(temporal_models):
        Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
        R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

        #find sequence label config. with highest prob.
        config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
        R_pred_config = model.feasible_y[config_index]
        R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

        #for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
        R_pred_probs = torch.FloatTensor(R_pred_config.shape)
        for idx in range(R_pred_config.shape[0]):
            R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

        R_pred_probs = R_pred_probs.numpy()
        R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]

        Li_dev = torch.LongTensor(MRI_data_temporal['Li_dev'].cpu().numpy())
        R_pred_frame = model.predict_proba(Li_dev.to(device)) #predict per sequence

        #find sequence label config. with highest prob.
        config_index = np.argmax(R_pred_frame.detach().cpu().numpy(), axis=1) 
        R_pred_config = model.feasible_y[config_index]
        R_pred_max = torch.FloatTensor(np.max(R_pred_frame.detach().cpu().numpy(), axis=1))

        #for each 1 in config, multiply by prob. label for sequence (1-prob label otherwise)
        R_pred_probs = torch.FloatTensor(R_pred_config.shape)
        for idx in range(R_pred_config.shape[0]):
            R_pred_probs[idx,:] = torch.LongTensor(R_pred_config[idx,:].cpu()).float()*R_pred_max[idx]

        R_pred_probs = R_pred_probs.numpy()
        R_pred_probs[R_pred_probs < 0] = 1+R_pred_probs[R_pred_probs < 0]
        R_pred_frame_label = np.round(R_pred_probs.ravel())
        R_pred_frame_label[R_pred_frame_label == 0.] = 2.

        scores = [iterations, seed, model]
        for metric in ['accuracy', 'f1', 'recall', 'precision']:
            score = metric_score(Y_dev.cpu(), R_pred_frame_label, metric)
            print(f"{metric.capitalize()}: {score:.3f}")
            
            scores.append(score)
        
        if best_class_balance == None or scores[4] > best[4]:
            best_class_balance = scores
        print()

print(best_class_balance)

1
Accuracy: 0.605
F1: 0.308
Recall: 0.500
Precision: 0.223

Accuracy: 0.684
F1: 0.319
Recall: 0.421
Precision: 0.257

Accuracy: 0.695
F1: 0.356
Recall: 0.479
Precision: 0.283

Accuracy: 0.781
F1: 0.421
Recall: 0.453
Precision: 0.394

Accuracy: 0.545
F1: 0.225
Recall: 0.375
Precision: 0.161

Accuracy: 0.426
F1: 0.230
Recall: 0.487
Precision: 0.150

Accuracy: 0.569
F1: 0.218
Recall: 0.342
Precision: 0.160

Accuracy: 0.421
F1: 0.237
Recall: 0.512
Precision: 0.155

Accuracy: 0.512
F1: 0.130
Recall: 0.207
Precision: 0.095

Accuracy: 0.666
F1: 0.309
Recall: 0.425
Precision: 0.243

10
Accuracy: 0.705
F1: 0.509
Recall: 0.868
Precision: 0.360

Accuracy: 0.738
F1: 0.542
Recall: 0.879
Precision: 0.391

Accuracy: 0.694
F1: 0.440
Recall: 0.683
Precision: 0.325

Accuracy: 0.812
F1: 0.594
Recall: 0.779
Precision: 0.479

Accuracy: 0.731
F1: 0.430
Recall: 0.576
Precision: 0.343

Accuracy: 0.598
F1: 0.371
Recall: 0.673
Precision: 0.256

Accuracy: 0.610
F1: 0.394
Recall: 0.720
Precision: 0.271

Accuracy:

In [154]:
one_best = best_class_balance